In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from time import sleep

driver = webdriver.Chrome()
driver.get('https://shopee.co.th/search?keyword=jewelry')
sleep(3)

thai_button = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

username = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/div[2]/div[1]/input')
password = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/div[3]/div[1]/input')
submit = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[2]/button')

username.send_keys('0988270432')
password.send_keys('@Piwat087')
sleep(3)

submit.send_keys(Keys.ENTER)
sleep(3)

page = 0
shp_product = []

print("Shopee In Progress ...")
while (page <= 16):
    
    driver.get(f'https://shopee.co.th/search?keyword=jewelry&page={page}')
    sleep(3)
    driver.execute_script("document.body.style.zoom='10%'")
    sleep(3)
    
    for item in driver.find_elements(By.CLASS_NAME, 'KMyn8J'):
        try:
            name = item.find_element(By.CLASS_NAME, 'FDn--\+').text
            price = item.find_element(By.CLASS_NAME, 'vioxXd.rVLWG6').text.split()
            quan = item.find_element(By.CLASS_NAME, 'r6HknA.uEPGHT').text.split()
            location = item.find_element(By.CLASS_NAME, 'zGGwiV').text.replace('จังหวัด', '')
            
            price = price[-1].replace('฿', '').replace(',', '')
            quan = quan[1].replace('.', '').replace('พัน', '00')
            shp_product.append((name, price, quan, int(price)*int(quan), location, 'Shopee'))
            
        except NoSuchElementException:
            pass

    page += 1
    print("Page", page, "complete :", len(shp_product), "Product")

print("Shopee Product list:", len(shp_product))
print("Shopee Completed !!!")
driver.close()

shp_products = pd.DataFrame(shp_product, columns = ['name', 'price', 'quantity', 'sales', 'location', 'platforms'])

pd.set_option('display.max_rows', None)
shp_products

Shopee In Progress ...
Page 1 complete : 60 Product
Page 2 complete : 119 Product
Page 3 complete : 177 Product
Page 4 complete : 227 Product
Page 5 complete : 284 Product
Page 6 complete : 340 Product
Page 7 complete : 395 Product
Page 8 complete : 450 Product
Page 9 complete : 509 Product
Page 10 complete : 567 Product
Page 11 complete : 623 Product
Page 12 complete : 681 Product
Page 13 complete : 741 Product
Page 14 complete : 798 Product
Page 15 complete : 856 Product
Page 16 complete : 913 Product
Page 17 complete : 973 Product
Shopee Product list: 973
Shopee Completed !!!


,name,price,quantity,sales,location,platforms
0,jewelry สร้อยคอแฟชั่น จี้เงิน สร้อยคอคนรัก สร้...,169,10800,1825200,สมุทรปราการ,Shopee
1,jewelry สร้อยข้อมือทองคำขาว สร้อยข้อมือแฟชั่น ...,159,5300,842700,สมุทรปราการ,Shopee
2,jewelry สุภาพสตรีสร้อยคอทองคำ 18k จี้ทองดีไซน์...,149,9900,1475100,สมุทรปราการ,Shopee
3,jewelry ชุด 2 แหวนทองคำขาว แหวนสุภาพสตรี แหวนเ...,169,1600,270400,สมุทรปราการ,Shopee
4,Memorine Jewelry แหวนเงินแท้ 925 ฝังเพชรสวิส(C...,420,2600,1092000,เชียงใหม่,Shopee
5,แหวน Exquisite Square Crystal Silver Ring Char...,16,152,2432,ต่างประเทศ,Shopee
6,Kind Jewelry เซตต่างหูสุดคุ้ม 2คู่ Pro emmie (...,1290,874,1127460,กรุงเทพมหานคร,Shopee
7,Haus of Jewelry CELESTIAL STUD EARRINGS ต่างหู...,580,40,23200,กรุงเทพมหานคร,Shopee
8,Louis Montini (Jewelry) Sterling Silver 925 Ea...,359,55,19745,กรุงเทพมหานคร,Shopee
9,Eramyth jewelry: ต่างหูห่วง ดีไซน์ลงสี White E...,659,40,26360,กรุงเทพมหานคร,Shopee


In [4]:
driver = webdriver.Chrome()
page = 1
laz_product = []

print("Lazada In Progress ...")
while (len(laz_product) <= len(shp_product)):
    
    driver.get(f'https://www.lazada.co.th/tag/jewelry/?spm=a2o4m.home.search.1.3c0e7f6dugi2So&q=jewelry&_keyori=ss&from=search_history&sugg=jewelry_0_1&catalog_redirect_tag=true&page={page}')
    sleep(3)
    driver.execute_script("document.body.style.zoom='10%'")
    sleep(3)
    
    for item in driver.find_elements(By.CLASS_NAME, 'buTCk'):
        try:
            name = item.find_element(By.CLASS_NAME, 'RfADt').text
            price = item.find_element(By.CLASS_NAME, 'ooOxS').text.split()
            quan = item.find_element(By.CLASS_NAME, '_1cEkb').text.split()
            location = item.find_element(By.CLASS_NAME, 'oa6ri ').text
            
            price = price[-1].replace('฿', '').replace(',' ,'')
            quan = quan[0].replace(',','').replace('k','000').replace('+','')
            laz_product.append((name, price, quan, float(price)*int(quan), location, 'Lazada'))
            
        except NoSuchElementException:
            pass
    
    print("Page", page, "complete !!!")
    page += 1

print("Lazada Product list:", len(laz_product))
print("Lazada Completed !!!")
driver.close()

laz_products = pd.DataFrame(laz_product,
                            columns=['name', 'price', 'quantity', 'sales', 'location', 'platforms'])
laz_products

Lazada In Progress ...
Page 1 complete !!!
Page 2 complete !!!
Page 3 complete !!!
Page 4 complete !!!
Page 5 complete !!!
Page 6 complete !!!
Page 7 complete !!!
Page 8 complete !!!
Page 9 complete !!!
Page 10 complete !!!
Page 11 complete !!!
Page 12 complete !!!
Page 13 complete !!!
Page 14 complete !!!
Page 15 complete !!!
Page 16 complete !!!
Page 17 complete !!!
Page 18 complete !!!
Page 19 complete !!!
Page 20 complete !!!
Page 21 complete !!!
Page 22 complete !!!
Page 23 complete !!!
Page 24 complete !!!
Page 25 complete !!!
Page 26 complete !!!
Page 27 complete !!!
Lazada Product list: 977
Lazada Completed !!!


,name,price,quantity,sales,location,platforms
0,LS ชุด 2 แหวนทองคำขาว แหวนสุภาพสตรี แหวนเงิน แ...,169.00,1776,300144.00,สมุทรปราการ,Lazada
1,LS สร้อยคอแฟชั่น จี้เงิน สร้อยคอคนรัก สร้อยคอผ...,169.00,2877,486213.00,สมุทรปราการ,Lazada
2,สร้อยโซ่เงินไขว้หัวใจผีเสื้อ Y2K 17กม. สำหรับเ...,10.67,6715,71649.05,ฉะเชิงเทรา,Lazada
3,สร้อยโซ่ประดับโบว์รูปหัวใจดอกทิวลิปติดปีก17กม....,12.48,1502,18744.96,ต่างประเทศ,Lazada
4,สร้อยโชคเกอร์ร้อยจี้ตาชั่วร้ายในแบบของคุณกำไลโ...,17.20,945,16254.00,ฉะเชิงเทรา,Lazada
5,36คู่เกาหลีแฟชั่นต่างหูผสมรูปแบบ Ins Geometirc...,29.00,9999,289971.00,ต่างประเทศ,Lazada
6,ต่างหูเงินทรงยาวทองคำ17กม. สำหรับผู้หญิงชุดต่า...,11.43,1595,18230.85,ฉะเชิงเทรา,Lazada
7,Vnox A-Z 26ตัวอักษรโรมันสร้อยคอสำหรับผู้หญิงสแ...,59.67,8306,495619.02,ฉะเชิงเทรา,Lazada
8,สร้อยจี้รูปหัวใจรูปโบว์หัวใจผีเสื้อหรูหรา17กม....,13.34,480,6403.20,ฉะเชิงเทรา,Lazada
9,แฟชั่นใหม่เงินU.S. Armyแหวนอัญมณีกองทัพแหวนสำห...,23.90,3665,87593.50,ฉะเชิงเทรา,Lazada


In [5]:
products = pd.concat([shp_products, laz_products])
products.to_csv('ShopeeLazadaJewelryProducts.csv')
products

,name,price,quantity,sales,location,platforms
0,jewelry สร้อยคอแฟชั่น จี้เงิน สร้อยคอคนรัก สร้...,169,10800,1825200.00,สมุทรปราการ,Shopee
1,jewelry สร้อยข้อมือทองคำขาว สร้อยข้อมือแฟชั่น ...,159,5300,842700.00,สมุทรปราการ,Shopee
2,jewelry สุภาพสตรีสร้อยคอทองคำ 18k จี้ทองดีไซน์...,149,9900,1475100.00,สมุทรปราการ,Shopee
3,jewelry ชุด 2 แหวนทองคำขาว แหวนสุภาพสตรี แหวนเ...,169,1600,270400.00,สมุทรปราการ,Shopee
4,Memorine Jewelry แหวนเงินแท้ 925 ฝังเพชรสวิส(C...,420,2600,1092000.00,เชียงใหม่,Shopee
5,แหวน Exquisite Square Crystal Silver Ring Char...,16,152,2432.00,ต่างประเทศ,Shopee
6,Kind Jewelry เซตต่างหูสุดคุ้ม 2คู่ Pro emmie (...,1290,874,1127460.00,กรุงเทพมหานคร,Shopee
7,Haus of Jewelry CELESTIAL STUD EARRINGS ต่างหู...,580,40,23200.00,กรุงเทพมหานคร,Shopee
8,Louis Montini (Jewelry) Sterling Silver 925 Ea...,359,55,19745.00,กรุงเทพมหานคร,Shopee
9,Eramyth jewelry: ต่างหูห่วง ดีไซน์ลงสี White E...,659,40,26360.00,กรุงเทพมหานคร,Shopee
